In [146]:
'''
Jiayou Qin 03/17/2024
SIFT based stablizer
a simple demo of stablization using sift
filtered sift anomaly with mean value of vectors
'''


import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
 
video_path = "stationary.mp4" #change this file name as needed
cap = cv.VideoCapture(video_path)


visual_intensity = 5
visualize_points = False
visualize_cursor = True

for i in range(1): #skip frames, must be larger than 1
    success, frame = cap.read()
# frame = cv.rotate(frame, cv.ROTATE_90_CLOCKWISE)
sift = cv.SIFT_create()
frame = cv.resize(frame,(800,600))
y,x = frame.shape[:2]
center = np.array([x/2,y/2], dtype=float)
cursor = center.copy()

last_frame = frame
kp, des = sift.detectAndCompute(frame,None)
cv.namedWindow('image',cv.WINDOW_NORMAL)
cv.imshow('image', frame)
cv.waitKey(0) #press key to begin
while True:
    for i in range(5):
        success, frame = cap.read()
    if not success:
        break
    delta = np.array([],dtype=float)
    visual_info = np.array([],dtype=float)
    frame = cv.resize(frame,(800,600))
#     frame = cv.rotate(frame, cv.ROTATE_90_CLOCKWISE)
    out_frame = frame.copy()
    if not success:
        break
    kp_,des_ = kp,des
    kp, des = sift.detectAndCompute(frame,None)

    bf = cv.BFMatcher()
    matches = bf.knnMatch(des,des_,k=2)

    # Apply ratio test
    good = []
    for m,n in matches:
        if m.distance < 0.75*n.distance:
            p1 = np.array(kp[m.queryIdx].pt, dtype=float)
            p2 = np.array(kp_[m.trainIdx].pt, dtype=float)
            delta = np.concatenate((delta,p2-p1))
            visual_info = np.concatenate((visual_info,p1,p2))
            
    delta = delta.reshape(-1,2)
    mean = np.mean(delta, axis=0)
    #eliminate anomaly for better accuracy
    threshold = 5 # anomaly higher than the threshold will be eliminated
    mask = np.absolute(delta)>np.absolute(mean*threshold)
    mask = np.logical_or(mask[:,0], mask[:,1])
    delta[mask] = mean
    mean = np.mean(delta, axis=0)
    #do not draw masked
    visual_info = visual_info.reshape(-1,2,2)
    visual_info[mask] = [[0,0],[0,0]]
    
    if visualize_points:
        for p1,p2 in visual_info:
            cv.arrowedLine(out_frame,p2.astype(int),p1.astype(int),(0,0,255),1)
    if visualize_cursor:
        vec = center + mean*visual_intensity
        cursor -= mean
        cv.arrowedLine(out_frame,center.astype(int),vec.astype(int),(0,255,0),3)
        cv.arrowedLine(out_frame,center.astype(int),cursor.astype(int),(0,0,255),3)
        cv.circle(out_frame, center.astype(int), 15, (0,255,255), 2)

    cv.imshow('image', out_frame)
    k = cv.waitKey(5)
    if k ==27:
        break
    
    last_frame = frame
    
#     break;
cap.release()
cv.destroyAllWindows()
